In [1]:
#import ipdb;ipdb.set_trace()

python -m torch.distributed.launch --nproc_per_node=1 --nnodes=1 byol_main.py

In [1]:
import torch
import tensorflow as tf
import numpy as np
target1 = torch.load('./rand/target1.pt').cpu().detach()
target2 = torch.load('./rand/target2.pt').cpu().detach()
pred1 = torch.load('./rand/pred1.pt').cpu().detach()
pred2 = torch.load('./rand/pred2.pt').cpu().detach()
tind1 = torch.load('./rand/tind1.pt').cpu().detach()
tind2 = torch.load('./rand/tind2.pt').cpu().detach()
pind1 = torch.load('./rand/pind1.pt').cpu().detach()
pind2 = torch.load('./rand/pind2.pt').cpu().detach()

num_rois =16
batch_size=128
temperature=0.1
max_val=1e9
infinity_proxy =1e9
import numpy as np
import torch
from torch import nn

def make_same_obj(ind_0, ind_1):
    b = ind_0.shape[0]
    same_obj = torch.eq(ind_0.reshape([b, num_rois, 1]),
                         ind_1.reshape([b, 1, num_rois]))
    return same_obj.float().unsqueeze(2)

def manual_cross_entropy(labels, logits, weight):
    ce = - weight * torch.sum(labels * torch.nn.functional.log_softmax(logits,dim = -1), dim=-1)
    return torch.mean(ce)

same_obj_aa = make_same_obj(pind1, tind1)
same_obj_ab = make_same_obj(pind1, tind2)
same_obj_ba = make_same_obj(pind2, tind1)
same_obj_bb = make_same_obj(pind2, tind2)

pred1 = torch.nn.functional.normalize(pred1,dim=-1)
pred2 = torch.nn.functional.normalize(pred2,dim=-1)
target1 = torch.nn.functional.normalize(target1,dim=-1)
target2 = torch.nn.functional.normalize(target2,dim=-1)

labels_local = torch.nn.functional.one_hot(torch.tensor(np.arange(batch_size))
                                           ,batch_size).unsqueeze(1).unsqueeze(3)

logits_aa = torch.einsum("abk,uvk->abuv", pred1, target1) / temperature
logits_bb = torch.einsum("abk,uvk->abuv", pred2, target2) / temperature
logits_ab = torch.einsum("abk,uvk->abuv", pred1, target2) / temperature
logits_ba = torch.einsum("abk,uvk->abuv", pred2, target1) / temperature

labels_aa = labels_local * same_obj_aa
labels_ab = labels_local * same_obj_ab
labels_ba = labels_local * same_obj_ba
labels_bb = labels_local * same_obj_bb

logits_aa = logits_aa - max_val * labels_local * same_obj_aa
logits_bb = logits_bb - max_val * labels_local * same_obj_bb
labels_aa = 0. * labels_aa
labels_bb = 0. * labels_bb

labels_abaa = torch.cat([labels_ab, labels_aa], axis=2)
labels_babb = torch.cat([labels_ba, labels_bb], axis=2)

labels_0 = torch.reshape(labels_abaa, [batch_size, num_rois, -1])
labels_1 = torch.reshape(labels_babb, [batch_size, num_rois, -1])

num_positives_0 = torch.sum(labels_0, axis=-1, keepdims=True)
num_positives_1 = torch.sum(labels_1, axis=-1, keepdims=True)

labels_0 = labels_0 / torch.max(num_positives_0, torch.ones(num_positives_0.shape))
labels_1 = labels_1 / torch.max(num_positives_1, torch.ones(num_positives_1.shape))

obj_area_0 = torch.sum(make_same_obj(pind1, pind1), axis=[2, 3])
obj_area_1 = torch.sum(make_same_obj(pind2, pind2), axis=[2, 3])

weights_0 = torch.greater(num_positives_0[..., 0], 1e-3).float()
weights_0 = weights_0 / obj_area_0
weights_1 = torch.greater(num_positives_1[..., 0], 1e-3).float()
weights_1 = weights_1 / obj_area_1

logits_abaa = torch.cat([logits_ab, logits_aa], axis=2)
logits_babb = torch.cat([logits_ba, logits_bb], axis=2)

logits_abaa = torch.reshape(logits_abaa, [batch_size, num_rois, -1])
logits_babb = torch.reshape(logits_babb, [batch_size, num_rois, -1])

loss_a = manual_cross_entropy(labels_0, logits_abaa, weights_0)
loss_b = manual_cross_entropy(labels_1, logits_babb, weights_1)
loss = loss_a + loss_b
print(loss,loss_a,loss_b)

tensor(3.0559) tensor(1.5296) tensor(1.5263)


In [2]:
logits_abaa.shape

torch.Size([128, 16, 4096])

In [5]:
torch.nn.functional.log_softmax(logits_abaa,dim = -1)

tensor([[[-8.4685e+00, -8.5617e+00, -8.2342e+00,  ..., -8.3697e+00,
          -8.4482e+00, -8.3696e+00],
         [-8.5290e+00, -8.4105e+00, -8.0042e+00,  ..., -8.3132e+00,
          -8.4086e+00, -8.3132e+00],
         [-8.4688e+00, -8.5615e+00, -8.2323e+00,  ..., -8.3689e+00,
          -8.4491e+00, -8.3688e+00],
         ...,
         [-8.5515e+00, -8.4852e+00, -7.9304e+00,  ..., -8.3183e+00,
          -8.3675e+00, -8.3181e+00],
         [-8.4694e+00, -8.5612e+00, -8.2298e+00,  ..., -8.3681e+00,
          -8.4499e+00, -8.3680e+00],
         [-8.5523e+00, -8.4849e+00, -7.9280e+00,  ..., -8.3180e+00,
          -8.3681e+00, -8.3178e+00]],

        [[-8.4233e+00, -8.5514e+00, -8.4502e+00,  ..., -8.4930e+00,
          -8.3708e+00, -8.4929e+00],
         [-8.4451e+00, -8.6734e+00, -8.4437e+00,  ..., -8.4095e+00,
          -8.3473e+00, -8.4095e+00],
         [-8.4868e+00, -8.5398e+00, -8.3218e+00,  ..., -8.4683e+00,
          -8.3699e+00, -8.4683e+00],
         ...,
         [-8.4452e+00, -8

In [6]:
target1 = torch.load('./rand/target1.pt').cpu().detach()
target2 = torch.load('./rand/target2.pt').cpu().detach()
pred1 = torch.load('./rand/pred1.pt').cpu().detach()
pred2 = torch.load('./rand/pred2.pt').cpu().detach()
tind1 = torch.load('./rand/tind1.pt').cpu().detach()
tind2 = torch.load('./rand/tind2.pt').cpu().detach()
pind1 = torch.load('./rand/pind1.pt').cpu().detach()
pind2 = torch.load('./rand/pind2.pt').cpu().detach()
target1 = target1.numpy()
target2 = target2.numpy()
pred1 = pred1.numpy()
pred2 = pred2.numpy()
tind1 = tind1.numpy()
tind2 = tind2.numpy()
pind1 = pind1.numpy()
pind2 = pind2.numpy()

def manual_cross_entropy(labels, logits, weight):
  ce = - weight * np.sum(labels * tf.nn.log_softmax(logits), axis=-1)
  return np.mean(ce)

def make_same_obj(ind_0, ind_1):
    same_obj = np.equal(ind_0.reshape([batch_size, num_rois, 1]),
                     ind_1.reshape([batch_size, 1, num_rois]))
    return np.expand_dims(same_obj.astype("float64"), axis=2)

same_obj_aa = make_same_obj(pind1, tind1)
same_obj_ab = make_same_obj(pind1, tind2)
same_obj_ba = make_same_obj(pind2, tind1)
same_obj_bb = make_same_obj(pind2, tind2)

# L2 normalize the tensors to use for the cosine-similarity
pred1 = tf.math.l2_normalize(pred1, axis=-1)
pred2 = tf.math.l2_normalize(pred2, axis=-1)
target1 = tf.math.l2_normalize(target1, axis=-1)
target2 = tf.math.l2_normalize(target2, axis=-1)

target1_large = target1
target2_large = target2

labels_local = np.expand_dims(np.expand_dims(tf.one_hot(np.arange(batch_size),batch_size), axis=2), axis=1)

# Do our matmuls and mask out appropriately.
logits_aa = np.einsum("abk,uvk->abuv", pred1, target1_large) / temperature
logits_bb = np.einsum("abk,uvk->abuv", pred2, target2_large) / temperature
logits_ab = np.einsum("abk,uvk->abuv", pred1, target2_large) / temperature
logits_ba = np.einsum("abk,uvk->abuv", pred2, target1_large) / temperature

labels_aa = labels_local * same_obj_aa
labels_ab = labels_local * same_obj_ab
labels_ba = labels_local * same_obj_ba
labels_bb = labels_local * same_obj_bb

logits_aa = logits_aa - infinity_proxy * labels_local * same_obj_aa
logits_bb = logits_bb - infinity_proxy * labels_local * same_obj_bb
labels_aa = 0. * labels_aa
labels_bb = 0. * labels_bb

labels_abaa = np.concatenate([labels_ab, labels_aa], axis=2)
labels_babb = np.concatenate([labels_ba, labels_bb], axis=2)

labels_0 = np.reshape(labels_abaa, [batch_size, num_rois, -1])
labels_1 = np.reshape(labels_babb, [batch_size, num_rois, -1])

num_positives_0 = np.sum(labels_0, axis=-1, keepdims=True)
num_positives_1 = np.sum(labels_1, axis=-1, keepdims=True)

labels_0 = labels_0 / np.maximum(num_positives_0, 1)
labels_1 = labels_1 / np.maximum(num_positives_1, 1)

obj_area_0 = tf.math.reduce_sum(make_same_obj(pind1, pind1),axis=[2, 3])
obj_area_1 = tf.math.reduce_sum(make_same_obj(pind2, pind2),axis=[2, 3])

weights_0 = np.greater(num_positives_0[..., 0], 1e-3).astype("float64")
weights_0 = weights_0 / obj_area_0
weights_1 = np.greater(num_positives_1[..., 0], 1e-3).astype("float64")
weights_1 = weights_1 / obj_area_1

logits_abaa = np.concatenate([logits_ab, logits_aa], axis=2)
logits_babb = np.concatenate([logits_ba, logits_bb], axis=2)

logits_abaa = np.reshape(logits_abaa, [batch_size, num_rois, -1])
logits_babb = np.reshape(logits_babb, [batch_size, num_rois, -1])

loss_a = manual_cross_entropy(labels_0, logits_abaa, weights_0)
loss_b = manual_cross_entropy(labels_1, logits_babb, weights_1)
loss = loss_a + loss_b
print(loss,loss_a,loss_b)

3.055886189682516 1.5296317285339436 1.5262544611485722


In [7]:
tf.nn.log_softmax(logits_abaa)

<tf.Tensor: shape=(128, 16, 4096), dtype=float64, numpy=
array([[[-8.46845501e+00, -8.56166970e+00, -8.23422962e+00, ...,
         -8.36965763e+00, -8.44822865e+00, -8.36956691e+00],
        [-8.52902360e+00, -8.41052498e+00, -8.00417946e+00, ...,
         -8.31320967e+00, -8.40859892e+00, -8.31321277e+00],
        [-8.46884644e+00, -8.56150585e+00, -8.23234910e+00, ...,
         -8.36891967e+00, -8.44912416e+00, -8.36882788e+00],
        ...,
        [-8.55152328e+00, -8.48516716e+00, -7.93039675e+00, ...,
         -8.31825586e+00, -8.36748172e+00, -8.31809582e+00],
        [-8.46943384e+00, -8.56124329e+00, -8.22976886e+00, ...,
         -8.36811071e+00, -8.44989287e+00, -8.36801892e+00],
        [-8.55232382e+00, -8.48492581e+00, -7.92799716e+00, ...,
         -8.31796521e+00, -8.36812813e+00, -8.31780577e+00]],

       [[-8.42326112e+00, -8.55138315e+00, -8.45024629e+00, ...,
         -8.49301083e+00, -8.37082769e+00, -8.49292202e+00],
        [-8.44513550e+00, -8.67339447e+00, -8.